In [1]:
!pip install accelerate bitsandbytes peft


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
"""pretrained model"""
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "AI-Sweden-Models/Llama-3-8B-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="float16",
    device_map={"": "cpu"}  # Force CPU usage
)

# Run inference
input_text = "What is the capital of Sweden?"
inputs = tokenizer(input_text, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is the capital of Sweden? -> Vad är huvudstaden i Sverige? -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm -> Stockholm


In [8]:
model.save_pretrained("./pretrained_llama3_8_instruct")
tokenizer.save_pretrained("./pretrained_llama3_8_instruct")

('./pretrained_llama3_8_instruct\\tokenizer_config.json',
 './pretrained_llama3_8_instruct\\special_tokens_map.json',
 './pretrained_llama3_8_instruct\\tokenizer.json')

In [9]:
"""load pretrained model"""
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

# Path to your saved model
model_path = "./pretrained_llama3_8_instruct" 

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Configure 4-bit quantization with CPU offloading
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 to save VRAM
    llm_int8_enable_fp32_cpu_offload=True  # Offload to CPU when needed
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quant_config,
    device_map="auto"
)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 